# Imports

In [1]:
# Camera libraries
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

# Servo librarie
from SCSCtrl import TTLServo

# Motor librarie
from jetbot import Robot

# OpenCV librarie
import cv2

# Utilities
import time
import numpy as np
import threading

Succeeded to open the port
Succeeded to change the baudrate


# Global variables and camera setup

In [2]:
# Width and Height of the screen
screen_w = 300
screen_h = 300

# Red
lower_red = np.array([160, 100, 100])
upper_red = np.array([180, 255, 255])

# Yellow/Orange
lower_yellow = np.array([0, 50, 0])
upper_yellow = np.array([30, 255, 255])

# Blue/Purpel
lower_blue = np.array([90, 100, 100])
upper_blue = np.array([120, 255, 200])

# Minimum size a contour need to be
min_cnt_size = 50

# X and Y coordinates the duplo bricks need to be at in order for the arm to grab it
grab_zone_x = 140
grab_zone_y = 240

# An margin of error for the x,y above so pixel coordinates don't have to aline 100% (That would be pretty hard)
error_margin = 10

# Instansiate the robot and a speed to move with
robot = Robot()
speed = 0.3

# Set the servos at initial position
TTLServo.servoAngleCtrl(1, 0, 1, 150)
TTLServo.servoAngleCtrl(2, 0, 1, 150)
TTLServo.servoAngleCtrl(3, 0, 1, 150)
TTLServo.servoAngleCtrl(4, 0, 1, 150)
TTLServo.servoAngleCtrl(5, 0, 1, 150)

512

We need to start the camera feed here as part of the variables, since we need to keep track of the number of threads that are running when we start.

In [3]:
# Create camere
camera = Camera.instance(width = screen_w, height = screen_h)

# Create widget
image_widget = ipywidgets.Image()

# Link the camera and widget
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# Display the camera feed using the widget
display(image_widget)

# We can now store all the active threads we are running
threads = threading.active_count()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# Functions

We are gonna need three functions for this project.
- A function that picks up a place the duplo bricks in the basket
- A function that locates the duplo bricks
- A function that drives to the duplo bricks

## grab

The grab functio will grab bricks in front of the robot then swing the arm to the back of the robot where a basket is attached and drop the bricks in the basket.

In [5]:
# Grap function
def grab():
    
    # GRAB
    # Position arm down
    TTLServo.xyInput(200, -90)
    time.sleep(1)
    # Close grip
    TTLServo.servoAngleCtrl(4, 40, -1, 150)
    time.sleep(5)
    
    # PLACE IN BASKET
    # Position arm at back
    TTLServo.servoAngleCtrl(2, 100, -1, 200)
    TTLServo.servoAngleCtrl(3, 100, 1, 200)
    time.sleep(5)
    # Open grib
    TTLServo.servoAngleCtrl(4, 100, 1, 150)
    time.sleep(3)
    # Position arm at initial position
    TTLServo.servoAngleCtrl(2, 0, 1, 200)
    TTLServo.servoAngleCtrl(3, 0, 1, 200)

## find_toy_pos

The find_toy_pos function takes a images from the camre and filters for different colours that the duplo bricks have. Then contours of the bricks are obtained and the brick with the biggest contour (we are asumening this means it is closest to the robot) are selected if it is bigger then the minimum size we have selected, and a bounding box is calculated and drawn on the input images aswell as the center of the bounding box. The center x,y is then returned. If no contour was obtained or the biggest counter found was samller then the minimum size '-1,-1' is returned insted.

In [15]:
def find_toy_pos(input_image):
    
    # Convert video frames to HSV color space.
    hsv = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV)
    
    # Create masks for pixels that match the target colors.
    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # Combining the masks
    result = mask_red + mask_yellow + mask_blue
    
    # Erode, this process will remove the relatively 
    # small area in the mask just selected, which can be understood as denoising.
    result = cv2.erode(result, None, iterations=1)

    # dilate, the corrosion process just now will cause the large area to become 
    # smaller and the small area to disappear. This step is to restore the large area to its previous size.
    result = cv2.dilate(result, None, iterations=1)
    
    # Obtain the conformed area contour.
    cnts = cv2.findContours(result.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
    
    # Check if there is any contours
    if len(cnts) > 0:
        
        # Get the biggest contour
        c = max(cnts, key=cv2.contourArea)
        
        # Check if the area is bigger than the minimum size
        if cv2.contourArea(c) > min_cnt_size:
            
            # Make a bounding box and get the values
            x, y, w, h = cv2.boundingRect(c)
            
            # Calculate center of the bounding box
            center_x = int(x + (w / 2))
            center_y = int(y + (h / 2))
            
            # Draw a circle at the center and the bounding box
            cv2.circle(input_image, (center_x, center_y), 2, (255, 255, 255), -1)
            cv2.rectangle(input_image, (x, y), (x + w, y + h), (255, 255, 255), 1)
            
            # Return the center x,y
            return center_x, center_y
    
    # else return -1,-1
        else:
            return -1, -1
    else:
        return -1, -1

## drive_to_position

The drive_to_position function checks where the x,y coordinates of the duplo brick, found with the find_toy_pos funtion, is in relationship to where it should be in order for the robot arm to grab it. We know (by trail and error) at what position the center of the brick should be at in the images (+/- an error margin) in order for the arm to grab it. When the brick is at the position it needs to be the funtion returns true.

For example, can we check if the bricks x coordinate is less than the position we want it to be at, the brick is to our left and we then turn the robot to the left.

In [16]:
def drive_to_position(input_image, zone_x, zone_y, center_x, center_y):
    
    #Left/Right
    
    # This means find_toy_pos() did not find any duplo bricks
    if center_x < 0:
        # Stop the robot, draw the grab zone
        robot.stop()
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (0, 0, 0), 1)
        
        # The brick is to the left
    elif center_x < zone_x - error_margin:
        # Truns left, draw the grab zone and write what turn we are making
        robot.left(speed)
        cv2.putText(input_image, 'Left', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (0, 0, 0), 1)
        
        # The brick is to the right
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (0, 0, 0), 1)
    elif center_x > zone_x + error_margin:
        # Truns left, draw the grab zone and write what turn we are making
        robot.right(speed)
        cv2.putText(input_image, 'Right', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (0, 0, 0), 1)  
    
    # Forward/Backward
    
    # This means find_toy_pos() did not find any duplo bricks
    if center_y < 0:
        # Stop the robot, draw the grab zone
        robot.stop()
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (0, 0, 0), 1)
        
        # The brick is in front of the the grab zone
    elif center_y < zone_y - error_margin:
        # Drives forward, draw the grab zone and write the direction
        robot.forward(speed)
        cv2.putText(input_image, 'Forward', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (0, 0, 0), 1)
        
        # The brick is behind the grab zone
    elif center_y > zone_y + error_margin:
        # Drives backwards, draw the grab zone and write the direction
        robot.backward(speed)
        cv2.putText(input_image, 'Backwards', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (0, 0, 0), 1)
        
        # The bricks center is in the grab zone
    if center_x > zone_x - error_margin and center_x < zone_x + error_margin and center_y > zone_y - error_margin and center_y < zone_y + error_margin:
        # Stop the robot, draw the grab zone, returns true
        robot.stop()
        cv2.circle(input_image, (zone_x, zone_y), error_margin, (255, 255, 255), 1)
        return True

    return False

## execute

The execute function basicly funtions as our main loop and ties all the functions together with the camera feed

In [8]:
# Practicly the "game loop"
def execute(change):
    # Global variable
    global grab_zone_x, grab_zone_y, grab_proccess, threads
    
    # Get the new images
    image = change['new']
    
    # Check if there is no more active threds than when we started
    if threading.active_count() <= threads:
        
        # Finds a duplo bricks cneter coordinated
        center_x, center_y = find_toy_pos(image)
        
        # Drive over to the brick
        in_position = drive_to_position(image, grab_zone_x, grab_zone_y, center_x, center_y)
        
        # If we are at teh brick
        if in_position:
            # Starte the grab function in a new thread. This is to avoid the camera feed from freezing.
            grab_thread = threading.Thread(target = grab)
            grab_thread.start()
        
    # Display the images
    image_widget.value = bgr8_to_jpeg(image)

# Start everything

Lastly we need to link the camera with the executa funtion.

In [9]:
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

# Stop everyrhing

In [17]:
camera.unobserve(execute, names='value')
time.sleep(1)
camera.stop()